# Imports & Device Setup

In [1]:
!pip install skorch braindecode moabb


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision
from torchvision.transforms import v2
from tqdm import tqdm
from skorch.helper import predefined_split

from skorch.callbacks import LRScheduler

import pandas as pd
import mne

np.int = int
np.bool = bool
np.object = object
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
    create_fixed_length_windows
)
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGConformer
from braindecode import EEGClassifier
from braindecode.datasets import BaseDataset, BaseConcatDataset, create_from_X_y


from braindecode.augmentation import (
    FTSurrogate,
    SmoothTimeMask,
    ChannelsDropout,
    AugmentedDataLoader
)

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/warren/UCLA/W24/ECEC147/project/venv/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print("Using", device)

Using cuda


# Important Variables

In [4]:
seed = 1
set_random_seeds(seed, str(device)=='cuda')
num_channels = 22
num_classes = 4
ch_names = [str(i) for i in range(num_channels)]
classes = list(range(num_classes))
input_window_samples = 800
lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64
n_epochs = 1000
folds = 10
sfreq = 250

# Load Data

In [5]:
X_train_valid = np.load("./project_data/X_train_valid.npy")
y_train_valid = np.load("./project_data/y_train_valid.npy") - 769

X_test = np.load("./project_data/X_test.npy")
y_test = np.load("./project_data/y_test.npy") - 769

person_train_valid = np.load("./project_data/person_train_valid.npy")
person_test = np.load("./project_data/person_test.npy")

In [6]:
indices = np.random.permutation(X_train_valid.shape[0])
split_idx = int(X_train_valid.shape[0] * ((folds-1)/folds))
X_train_valid = X_train_valid[:,:,0:800]
X_test = X_test[:,:,0:800]

X_train, X_valid = X_train_valid[indices[:split_idx]], X_train_valid[indices[split_idx:]]
y_train, y_valid = y_train_valid[indices[:split_idx]], y_train_valid[indices[split_idx:]]

## Create Braindecode Datasets

In [ ]:
train_dataset = create_from_X_y(X_train, y_train, False, sfreq, ch_names=ch_names)
valid_dataset = create_from_X_y(X_valid, y_valid, False, sfreq, ch_names=ch_names)
test_dataset = create_from_X_y(X_test, y_test, False, sfreq, ch_names=ch_names)

Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.
Creating RawArray with float64 data, n_channels=22, n_times=800
    Range : 0 ... 799 =      0.000 ...     3.196 secs
Ready.


# Preprocessing

In [ ]:
low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
factor_new = 1e-3
init_block_size = 800

preprocessors = [
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz, picks=ch_names, verbose=False),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
        picks=ch_names,
    ),
]

# Preprocess the data
# preprocess(train_dataset, preprocessors)
# preprocess(valid_dataset, preprocessors)
# preprocess(test_dataset, preprocessors)

# Augmentations

In [ ]:
# channels_dropout = ChannelsDropout(
#     probability=0.5,
#     p_drop=1
# )

# smooth_time_mask = SmoothTimeMask(
#     probability=0.5,
#     mask_len_samples=300
# )

# transforms = [smooth_time_mask, channels_dropout]

# Model Training

In [ ]:
model = ShallowFBCSPNet(
    num_channels,
    num_classes,
    n_times=input_window_samples,
    final_conv_length="auto",
)
print(model)
model.to(device)

print(f'TRAIN LENGTH: {len(train_dataset)}')
print(f'VALID LENGTH: {len(valid_dataset)}')
print(f'TEST LENGTH: {len(test_dataset)}')

In [ ]:
clf = EEGClassifier(
    model,
    iterator_train=AugmentedDataLoader,
    iterator_train__transforms=transforms,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_dataset),
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    device=device,
    classes=classes,
    max_epochs=n_epochs,
)
clf.fit(train_dataset, y=None)

# evaluated the model after training
y_test = test_dataset.get_metadata().target
test_acc = clf.score(test_dataset, y=y_test)
print(f"Test acc: {(test_acc * 100):.2f}%")